In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

In [3]:
DATASET_PATH = "."  # Change this if your path is different
IMAGE_SIZE = 128  # Better accuracy with larger images
MAX_IMAGES = 1000  # Adjust as per your dataset

In [5]:
def load_hog_features(base_path, image_size=128, max_images_per_class=1000):
    X = []
    y = []
    classes = ["Cat", "Dog"]
    
    for label, class_name in enumerate(classes):
        folder_path = os.path.join(base_path, class_name)
        image_files = os.listdir(folder_path)[:max_images_per_class]
        
        print(f"📥 Loading {class_name} images: ")
        for img_name in tqdm(image_files):
            img_path = os.path.join(folder_path, img_name)
            try:
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (image_size, image_size))
                
                # Extract HOG features
                features = hog(
                    img,
                    orientations=9,
                    pixels_per_cell=(16, 16),
                    cells_per_block=(2, 2),
                    block_norm='L2-Hys',
                    feature_vector=True
                )
                X.append(features)
                y.append(label)
            except Exception as e:
                continue
    
    return np.array(X), np.array(y)


In [7]:
X, y = load_hog_features(DATASET_PATH, image_size=IMAGE_SIZE, max_images_per_class=MAX_IMAGES)

print("✅ HOG Features shape:", X.shape)
print("✅ Labels shape:", y.shape)


📥 Loading Cat images: 


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 107.16it/s]


📥 Loading Dog images: 


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 106.85it/s]


✅ HOG Features shape: (1992, 1764)
✅ Labels shape: (1992,)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [11]:
model = LinearSVC(C=1.0, max_iter=10000)
print("🧠 Training Linear SVM...")
model.fit(X_train_scaled, y_train)
print("✅ Training complete!")


🧠 Training Linear SVM...


C:\Users\AKSHAY KUMAR\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


✅ Training complete!


In [13]:
y_pred = model.predict(X_test_scaled)

acc = accuracy_score(y_test, y_pred)
print(f"\n🎯 Accuracy: {acc * 100:.2f}%")

print("\n📊 Classification Report:\n")
print(classification_report(y_test, y_pred, target_names=["Cat", "Dog"]))



🎯 Accuracy: 64.16%

📊 Classification Report:

              precision    recall  f1-score   support

         Cat       0.64      0.67      0.65       200
         Dog       0.65      0.62      0.63       199

    accuracy                           0.64       399
   macro avg       0.64      0.64      0.64       399
weighted avg       0.64      0.64      0.64       399

